In [ ]:
!pip install PyMuPDF
!pip install pdfplumber
!pip install sentence-transformers
!pip install transformers
!pip install torch
!pip install faiss-cpu
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Upload the PDF file
uploaded = files.upload()

# Check the uploaded file name
for file_name in uploaded.keys():
    print(f'Uploaded file: {file_name}')

Saving Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf to Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf
Uploaded file: Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf


In [ ]:
# Step 1: Import libraries
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import pandas as pd
from google.colab import files

In [ ]:
# Step 2: Upload the PDF file
uploaded = files.upload()

# Get the uploaded PDF file name
pdf_path = next(iter(uploaded))  # Get the first uploaded file

Saving Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf to Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018 (3).pdf


In [ ]:
# Step 3: PDF Extraction
def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

In [ ]:

# Step 4: Chunking & Embedding
def chunk_and_embed(text):
    sentences = text.split('\n')
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can also use other Sentence-BERT models
    embeddings = model.encode(sentences)
    return sentences, embeddings

In [ ]:

# Step 5: Storing in FAISS Vector Database
def store_embeddings(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

In [ ]:

# Step 6: Query Processing
def process_query(query, index, sentences, model):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k=5)  # Retrieve top 5 most relevant chunks
    retrieved_chunks = [sentences[i] for i in indices[0]]
    return retrieved_chunks


In [ ]:

# Step 7: Generate Response using Hugging Face model
def generate_response(query, retrieved_chunks):
    context = "\n".join(retrieved_chunks)
    generator = pipeline('text-generation', model='gpt2')  # You can use other models like T5, GPT-J, etc.
    response = generator(f"Based on the following data, answer the user's query: {query}\n\n{context}", max_length=150)
    return response[0]['generated_text']

# Main pipeline
text = extract_pdf_text(pdf_path)
sentences, embeddings = chunk_and_embed(text)
index = store_embeddings(embeddings)

# Query handling
query = "What is the unemployment rate for a Master's degree?"
retrieved_chunks = process_query(query, index, sentences, SentenceTransformer('all-MiniLM-L6-v2'))
response = generate_response(query, retrieved_chunks)
print(response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the following data, answer the user's query: What is the unemployment rate for a Master's degree?

Source: U.S. Bureau of Labor Statistics
Table of Yearly U.S. GDP by 
Yearly Total GDP
2015 GDP for all industries was $31.397023; looking at the graph, 
Yearly Total GDP (in trillions of dollars) is calculated as the sum of the estimated cost of a high-paying high-school science degree (2.29 g/yr) vs. its estimated cost of attending a college without the degree (1.04 g/yr):

In addition, the U.S. economy added more low-skill jobs, led by


In [ ]:
# Query handling
query = "What is the unemployment rate for a Bachelor's degree?"
retrieved_chunks = process_query(query, index, sentences, SentenceTransformer('all-MiniLM-L6-v2'))
response = generate_response(query, retrieved_chunks)
print(response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the following data, answer the user's query: What is the unemployment rate for a Bachelor's degree?

Source: U.S. Bureau of Labor Statistics
Table of Yearly U.S. GDP by 
Industry (in millions of dollars)
Year
All Industries (%)

Employment

Rate

For every full-time worker, there are

18,951 unemployed (40.2%)

12,912 with an average age under 24 (25.3%)

1,977 unemployed (41.0%) (38.1%) 1,843 with a bachelor's degree (27.6%)

4,841 without a degree (19


In [2]:
# Install required libraries
!pip install PyMuPDF sentence-transformers faiss-cpu transformers pdfplumber google-colab

import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from google.colab import files

# Step 1: Upload the PDF
uploaded = files.upload()

# Get the uploaded PDF file name
pdf_path = next(iter(uploaded))  # Get the first uploaded file

# Step 2: PDF Extraction (Extracting text from the uploaded PDF)
def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")  # Extract text from each page
    return text

# Extracting text from the PDF
pdf_text = extract_pdf_text(pdf_path)

# Step 3: Chunking and Embedding the PDF Text
def chunk_and_embed(text):
    sentences = text.split('\n')
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence Transformer model for embeddings
    embeddings = model.encode(sentences)
    return sentences, embeddings

sentences, embeddings = chunk_and_embed(pdf_text)

# Step 4: Storing embeddings in FAISS for efficient retrieval
def store_embeddings(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance metric
    index.add(embeddings)
    return index

index = store_embeddings(embeddings)

# Step 5: Process Query and Search for Relevant Information
def process_query(query, index, sentences, model):
    query_embedding = model.encode([query])  # Convert query to embedding
    distances, indices = index.search(query_embedding, k=5)  # Retrieve top 5 relevant chunks
    retrieved_chunks = [sentences[i] for i in indices[0]]
    return retrieved_chunks

# Step 6: Generate a Response using Hugging Face Model
def generate_response(query, retrieved_chunks):
    context = "\n".join(retrieved_chunks)
    generator = pipeline('text-generation', model='gpt2')  # You can use other models like T5, GPT-J, etc.
    response = generator(f"Answer the following question based on the provided text: {query}\n\nContext: {context}", max_length=150)
    return response[0]['generated_text']

# Step 7: Chatbot Interface
def chat_with_pdf():
    print("Chat with PDF: Type 'exit' to quit.")
    while True:
        query = input("Ask a question: ")
        if query.lower() == 'exit':
            print("Exiting chat.")
            break

        retrieved_chunks = process_query(query, index, sentences, SentenceTransformer('all-MiniLM-L6-v2'))
        response = generate_response(query, retrieved_chunks)
        print(f"Answer: {response}")

# Start the chatbot
chat_with_pdf()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Saving Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf to Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: Answer the following question based on the provided text: Whats the unemployment rate of Master's degree

Context: Source: U.S. Bureau of Labor Statistics
2015 GDP for all industries was $31.397023; looking at the graph, 
Yearly Total GDP
Table of Yearly U.S. GDP by 
Yearly Total GDP (in trillions of dollars) _______________________________________________________________________

This information may become outdated at any time.

A NOTE TO FABLY READERS

This is not a substitute for being a good reader of a written publication or writing about an important subject. Do not read all of this in one go. Read it in one sitting for a long time, and then you
Ask a question: exit
Exiting chat.
